<a href="https://colab.research.google.com/github/Srihithabatchu/pdf-question-answer-bot/blob/main/Zidio_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Transfer Learning using ResNet50 ---
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load dataset (CIFAR-10 for simplicity)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = tf.image.resize(x_train, (224, 224))
x_test = tf.image.resize(x_test, (224, 224))
x_train, x_test = x_train / 255.0, x_test / 255.0

# Base Model (pretrained)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # freeze layers

# Add custom layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

# Fine-tuning
base_model.trainable = True
model.compile(optimizer=Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_ft = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


In [ ]:
# --- Autoencoder for Image Denoising ---
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Add Gaussian noise
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * np.random.normal(size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Build Autoencoder
input_img = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2,2), padding='same')(x)
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2,2), padding='same')(x)

x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2,2))(x)
decoded = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train_noisy, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

# Predictions
decoded_imgs = autoencoder.predict(x_test_noisy)

# Visualize
n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
    ax = plt.subplot(3, n, i+1)
    plt.imshow(x_test[i])
    plt.axis('off')
    if i == 0: plt.ylabel("Original")

    ax = plt.subplot(3, n, i+1+n)
    plt.imshow(x_test_noisy[i])
    plt.axis('off')
    if i == 0: plt.ylabel("Noisy")

    ax = plt.subplot(3, n, i+1+2*n)
    plt.imshow(decoded_imgs[i])
    plt.axis('off')
    if i == 0: plt.ylabel("Denoised")
plt.show()


In [ ]:
# --- RNN for Twitter US Airline Sentiment ---
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# Load dataset
dataset = tf.keras.utils.get_file(
    "airline_sentiment.csv",
    "https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv"
)
import pandas as pd
data = pd.read_csv(dataset)
data = data[['text', 'airline_sentiment']]

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['label'] = le.fit_transform(data['airline_sentiment'])

# Split data
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'].values, data['label'].values, test_size=0.2, random_state=42)

# Text vectorization
max_tokens = 10000
max_len = 100
vectorizer = layers.TextVectorization(max_tokens=max_tokens, output_sequence_length=max_len)
vectorizer.adapt(train_texts)

train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels)).batch(32).map(lambda x, y: (vectorizer(x), y))
test_ds = tf.data.Dataset.from_tensor_slices((test_texts, test_labels)).batch(32).map(lambda x, y: (vectorizer(x), y))

# Build model
model = models.Sequential([
    layers.Embedding(max_tokens, 128),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_ds, validation_data=test_ds, epochs=5)

# Plot accuracy/loss
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.title('Loss')
plt.show()

# Test on custom tweets
sample_tweets = ["I love flying with Delta!", "Worst flight ever.", "It was okay, nothing special."]
preds = model.predict(vectorizer(sample_tweets))
for t, p in zip(sample_tweets, preds):
    print(f"{t} → {le.inverse_transform([p.argmax()])[0]}")


In [ ]:
# --- Transfer Learning for Plant Disease Classification ---
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Use TensorFlow Flowers dataset as plant proxy
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)

# Image generator
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_gen = datagen.flow_from_directory(data_dir, target_size=(224,224), subset='training', batch_size=32)
val_gen = datagen.flow_from_directory(data_dir, target_size=(224,224), subset='validation', batch_size=32)

# Pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_gen, validation_data=val_gen, epochs=5)

# Fine-tuning
base_model.trainable = True
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_ft = model.fit(train_gen, validation_data=val_gen, epochs=3)

# Evaluate
loss, acc = model.evaluate(val_gen)
print(f"Validation Accuracy after fine-tuning: {acc:.4f}")
